# American Options Pricing Analytics
------------------
> **Idriss Afra**

This project aims to price American options using the Binomial model, the Barone-Adesi & Whaley model, and the Least-Squares MC method.

## The Binomial Model

The Binomial model is relatively easy to understand and implement as it assumes that the underlying asset $S_t$ moves up or down by a fixed percentage each period (Which may not be realistic...).

Let's define the movement factors $u$ and $d$ as following :
$$
u = e^{\sigma\sqrt{\Delta t}}\qquad d = e^{-\sigma\sqrt{\Delta t}} \\
$$
Where $\sigma$ is the volatility and $Δt$ is a tiny time step to maturity $T$. The risk-neutral probability under this model is :

$$
p = \frac{e^{(r-q)\Delta t}-d}{u-d} \\
$$
So that :
$$
E(S_{t+\Delta t}) = p × u × S_t + (1 - p) × d × S_t  
= p × S_{t+\Delta t}^u + (1 - p) × S_{t+\Delta t}^d \\
$$

Where $r$ is the zero-coupon interest rate and $q$ the dividend yield.

Once the Binomial tree simulation is done, the options are priced using a backward method :

* European Style :  
$$
PV_t(S_t) = e^{-r\Delta t}\left(p\times PV_{t+\Delta t}(S_{t+\Delta t}^u)+(1-p)\times PV_{t+\Delta t}(S_{t+\Delta t}^d)\right)
$$

* American Style :    
$$
PV_t(S_t) = max\left(S_t - K, e^{-r\Delta t}\left(p\times PV_{t+\Delta t}(S_{t+\Delta t}^u)+(1-p)\times PV_{t+\Delta t}(S_{t+\Delta t}^d)\right)\right) \\
$$

With the final condition : $ PV_T(S) = Max\left(ϕ × (S - K), 0 \right)  $

Where $ϕ = 1$ for call options and $ϕ = -1$ for put options, and $K$ is the strike price.

The accuracy of the binomial model improves with the number of time steps. However, increasing the number of time steps elevates the complexity and calculation time.

In [1]:
import numpy as np
import math

class binomial_model :
    """
    The Binomial model class.
    """
    def __init__(self, n_steps=10**4):
        """
        Init method.
        n_steps : number of time steps
        """
        self.n_steps = n_steps
        self.set_data = False

    def set_option_data(self, S0, T, vol, r, q=0):
        """
        Setter of option data.
        S0 : spot price
        T : maturity
        vol : ATM implied volatility
        r : zero-coupon interest rate
        q : dividend yield
        """
        self.S0 = S0
        self.T = T
        self.vol = vol
        self.r = r
        self.q = q
        self.set_data = True

    def spot_simulation(self):
        """
        The spot simulation under the Binomial model.
        The function returns the Binomial tree.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        # UP & DOWN Factors
        dt = self.T / self.n_steps
        u = math.exp(self.vol * math.sqrt(dt))
        d = 1. / u

        # Spot simulation : Binomial tree
        s = np.zeros((self.n_steps+1, self.n_steps+1))
        s[0,0] = self.S0
        for i in range(1, self.n_steps+1) :
            s[:i,i] = s[:i,i-1] * u
            s[i,i] = s[i-1,i-1] * d

        return s

    def binomial_price(self, flavor, K, amer=True):
        """
        The function returns the Binomial price.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        # Spot simulation : Binomial tree
        dt = self.T / self.n_steps
        u = math.exp(self.vol * math.sqrt(dt))
        d = 1. / u
        p = (math.exp((self.r - self.q) * dt) - d) / (u - d)
        s = np.zeros((self.n_steps+1, self.n_steps+1))
        s[0,0] = self.S0
        for i in range(1, self.n_steps+1) :
            s[:i,i] = s[:i,i-1] * u
            s[i,i] = s[i-1,i-1] * d

        # Option payoff at maturity
        phi = 1 if flavor.upper() == "CALL" else -1
        v = np.maximum(phi * (s[:, self.n_steps] - K), 0.)

        # Discount between 2 time steps
        discount = math.exp(-self.r * dt)

        # Backward loop
        for i in range(self.n_steps - 1, -1, -1) : # => i = n_steps-1 ... 0
            n_nodes = i + 1 # i+1 nodes at time step #i
            v = discount * (p * v[:n_nodes] + (1 - p) * v[1:n_nodes+1])
            if amer :
                # Possibility of early exercise
                v = np.maximum(phi * (s[:n_nodes, i] - K), v)

        return v[0]


## The Barone-Adesi & Whaley Model

The B.A.W is an adjusted version of the Black-Scholes model used to price American options. Its methodology is based on finding the exercise frontiers and computing the early-exercise premiums.

An approximation is made within this model making it less accurate for mid-maturity options and more accurate for short and long maturity options.

Let : $M = \frac{2r}{σ^2}$, and $N = \frac{2(r-q)}{σ^2}$.

The B.A.W price of an option of strike $K$ and maturity $T$ is :
\begin{equation}
\begin{split}
PV_{B.A.W}^{Amr}(K, T) & = PV_{Black}^{Eur}(K, T) + A × \left( \frac{S}{S*} \right)^b;\space ϕ ×  (S - S*) < 0 \\
PV_{B.A.W}^{Amr}(K, T) & = ϕ \times (S - K);\space ϕ ×  (S - S*) \ge 0
\end{split}
\end{equation}

Where :    
- $A = ϕ \times \left( \frac{S*}{q} \right) × \left( 1 - e^{-qT} N_{(0, 1)} \left(ϕ \times d_1(S*) \right) \right)$

- $d_1(S) = \left(log\left(\frac{S}{K}\right) + \left(r - q + 0.5 σ\sqrt T\right)  \right) / σ\sqrt T$

- $b = 0.5 × \left(1 - N + ϕ × \sqrt{(1 - N)^2 + 4M/K}  \right)$

- $S*$ represent the "Optimal Spot" (Exercise Frontier) and is the solution of the following equation :

$$
ϕ × (S - K) = PV_{Black}^{Eur}(K, T) + ϕ × \left( 1 - e^{-qT} N_{(0, 1)} \left(ϕ \times d_1(S) \right) \right) × S / q
$$

Solving the equation of $S*$ requires numerical methods. In this project, we combine both the Bisection and the Newton-Raphson methods.

More details can be found in [Barone-Adesi-Whaley 1987](https://github.com/Idriss-Afra/American-Options-Pricing-Analytics/blob/main/Barone-Adesi-Whaley%201987.pdf).

In [2]:
from scipy.stats import norm
import sys

class baw_model :
    """
    The Barone-Adhesi & Whaley model class.
    """
    def __init__(self):
        """
        Init method.
        """
        self.set_data = False

    def set_option_data(self, S0, T, vol, r, q=0):
        """
        Set option data method
        S0 : spot price
        T : maturity
        vol : ATM implied volatility
        r : zero-coupon interest rate
        q : dividend yield
        """
        self.S0 = S0
        self.T = T
        self.vol = vol
        self.r = r
        self.q = q
        self.set_data = True

    def d1(self, S, K):
        """
        The Black-Scholes d1 formula.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        v2T = self.vol**2 * self.T
        return (np.log(S / K) + (self.r - self.q) * self.T + v2T / 2) / v2T**0.5

    def black_price(self, flavor, S, K):
        """
        The Black-Scholes price.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        v2T = self.vol**2 * self.T
        d_1 = self.d1(S, K)
        d_2 = d_1 - v2T**0.5
        phi = 1 if flavor.upper() == "CALL" else  -1
        return phi * (S * math.exp(-self.q * self.T) * norm.cdf(phi * d_1) - K * math.exp(-self.r * self.T) * norm.cdf(phi * d_2))

    def abs_black_delta(self, flavor, S, K):
        """
        The Black-Scholes absolute delta.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        phi = 1 if flavor.upper() == "CALL" else  -1
        return math.exp(-self.q * self.T) * norm.cdf(phi * self.d1(S, K))

    def american_adj(self, flavor, S, S_optimal, K):
        """
        The American adjustment formula such as : American_Black_Price = European_Black_Price + American_Adj.

        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        phi = 1 if flavor.upper() == "CALL" else  -1
        M = 2 * self.r / self.vol**2
        N = 2 * (self.r - self.q) / self.vol**2
        a = 1 - math.exp(-self.r * self.T)
        b = 0.5 * (1 - N + phi * math.sqrt((1 - N)**2 + 4 * M / a))
        return phi * (S_optimal / b) * (1 - self.abs_black_delta(flavor, S_optimal, K)) * (S / S_optimal)**b

    def obj_func(self, flavor, S, K):
        """
        The Optimal Spot S* Equation.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        phi = 1 if flavor.upper() == "CALL" else  -1
        return self.black_price(flavor, S, K) + self.american_adj(flavor, S, S, K) - phi * (S - K)

    def optimal_spot_bisection(self, flavor, K) :
        """
        Bisection method to find the Optimal Spot S*.
        """
        # Number of iteration
        n_max = 750
        # Stopping error
        eps = 1e-4
        # Computation of S_min and S_max
        n_stdev = 3
        Fwd = self.S0 * math.exp((self.r - self.q) * self.T)
        S_min = Fwd / math.exp(n_stdev * self.vol * math.sqrt(self.T))
        S_max = Fwd * math.exp(n_stdev * self.vol * math.sqrt(self.T))
        # Choose the adequate numerical algorithm : Bisection or Newton-Raphson
        if self.obj_func(flavor, S_min,  K) * self.obj_func(flavor, S_max,  K) > 0:
            return self.optimal_spot_newton(flavor, K)

        n = 1
        while n <= n_max :
            optimal_spot = (S_min + S_max)  / 2
            func = self.obj_func(flavor, optimal_spot,  K)
            if (func == 0) or (abs(S_max - S_min) < eps) :
                return optimal_spot
            if func < 0 :
                S_min = optimal_spot
            else :
                S_max = optimal_spot
            n += 1

        return sys.exit("The American volatility calibration algorithm failed to converge. Please review data of the " + flavor.upper() + " with Strike " + str(K))

    def obj_func_deriv(self, flavor, S, K):
        """
        The derivative of the Optimal Spot S* Equation.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        phi = 1 if flavor.upper() == "CALL" else  -1
        M = 2 * self.r / self.vol**2
        N = 2 * (self.r - self.q) / self.vol**2
        a = 1 - math.exp(-self.r * self.T)
        b = 0.5 * (1 - N + phi * math.sqrt((1 - N)**2 + 4 * M / a))
        delta = phi * self.abs_black_delta(flavor, S, K)

        return delta * (1 - 1 / b) + phi * (1 / b) * (1 - phi * math.exp(-self.q * self.T) * norm.pdf(phi * self.d1(S, K)) / (self.vol * math.sqrt(self.T))) - phi

    def optimal_spot_newton(self, flavor, K):
        """
        Newton-Raphson method to find the Optimal Spot S*.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        # Number of iteration
        n_max = 750
        # Stopping error
        eps = 1e-4
        # Initial guess : Spot price
        optimal_spot = self.S0
        func = self.obj_func(flavor, optimal_spot,  K)
        func_deriv = self.obj_func_deriv(flavor, optimal_spot,  K)
        next_optimal_spot = optimal_spot - func / func_deriv

        nb_iteration = 1
        while abs(next_optimal_spot - optimal_spot) > eps :
            optimal_spot = next_optimal_spot
            func = self.obj_func(flavor, optimal_spot, K)
            func_deriv = self.obj_func_deriv(flavor, optimal_spot, K)
            next_optimal_spot = optimal_spot - func / func_deriv

            error = sys.exit("The algorithm failed to converge. Please review the input data.") if nb_iteration > n_max else "No error."
            nb_iteration += 1
        return next_optimal_spot

    def baw_price(self, flavor, K):
        """
        The B.A.W price.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        if (flavor.upper() == "CALL" and self.r > 0 and self.q == 0):
            # If the underlying does not pay dividends, then the early exercise of a call option in no more advantageous
            return self.black_price(flavor, self.S0, K)
        else :
            phi = 1 if flavor.upper() == "CALL" else  -1
            S_optimal = self.optimal_spot_bisection(flavor, K)
            #print("The Optimal Spot Price : ", round(S_optimal, 2))
            exercise_cond = phi * (self.S0 - S_optimal) >= 0
            return phi * (self.S0 - K) if exercise_cond else self.black_price(flavor, self.S0, K) + self.american_adj(flavor, self.S0, S_optimal, K)

## The Least-Squares Monte-Carlo Method

The Least-Squares Monte-Carlo method was designed by Longstaff & Schwartz. It is a very interesting and a brilliant pricing method for American and Bermudan payoffs. However, it relies on heavy numerical algorithms which increases its instability and complexity.

First, the spot prices are simulated $M$ times via the Black solution :    

$$
S_{t+dt} = S_{t} \times exp\left(\left(r - \frac{\sigma^2}{2}\right) dt + \sigma dW_t \right)
$$

Where : $dt=\frac{T}{N}$, $N$ the number of time steps, and $(W_t)_t$ a serie of standard brownian motions.

Then, the American options are priced using a backward loop of the following steps :

1 - Perform a regression method between spot prices simulated at step $t$ and discounted option prices computed at step $t + dt$. In this project, we use a $19$ degree ($20$ coefficients) polynomial regression for $M=6×10^5$ and compute the coefficients $(a_i)_i$ such as :

$$
∀j∈[1,\space 60000], \space  DF_{t,\space t+dt} × PV_{t+dt}(S^{j}_{t+dt}) = ∑_{i=0}^{19}a_i× \left(S^{j}_{t}\right)^i
$$

2 - Compute the "Continuation" price :

$$
 C_{t}\left(S^{j}_{t}\right) = ∑_{i=0}^{19}a_i×\left(S^{j}_{t}\right)^i
$$

3 - Compute the option prices at step $t$ considering the possibility of early exercise :

\begin{equation}
\begin{split}
PV_{t}\left(S^{j}_{t}\right) & = DF_{t,\space t+dt} × PV_{t+dt}\left(S^{j}_{t+dt}\right); \space  C_{t}\left(S^{j}_{t}\right) > ϕ × \left(S^{j}_{t} - K\right) \\ \\
& = ϕ × \left(S^{j}_{t} - K\right);\space C_{t}\left(S^{j}_{t}\right) \le ϕ × \left(S^{j}_{t} - K\right)
\end{split}
\end{equation}

With the final condition : $ PV_T(S) = Max\left(ϕ × (S - K), 0 \right) $

More details can be found in [Longstaff-Schwartz Least-Squares MC Approach](https://github.com/Idriss-Afra/American-Options-Pricing-Analytics/blob/main/Longstaff-Schwartz%20Least-Squares%20MC%20Approach.pdf).






In [3]:
import warnings
warnings.simplefilter('ignore', np.RankWarning)

class ls_mc :
    """
    The Least-Squares Monte-Carlo Method For American Options.
    """
    def __init__(self, Nsimul=6*10**5, Nsteps=100):
        """
        Init method.
        Nsimul : number of simulations
        Nsteps : number of time steps
        """
        self.Nsimul = Nsimul
        self.Nsteps = Nsteps
        self.set_data = False

    def set_option_data(self, S0, T, vol, r, q=0):
        """
        Setter of option data.
        S0 : spot price
        T : maturity
        vol : ATM implied volatility
        r : zero-coupon interest rate
        q : dividend yield
        """
        self.S0 = S0
        self.T = T
        self.vol = vol
        self.r = r
        self.q = q
        self.set_data = True

    def set_path(self) :
        """
        Monte-Carlo Simulation under the Black-Scholes model : Nsteps x Nsimul.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        # Generate random N(0,1) (Nb of time steps x Nb of simulations)
        normal = np.random.normal(0, 1, (self.Nsteps, self.Nsimul))

        # Time steps vector
        self.dt = self.T / self.Nsteps
        t = np.linspace(0, self.T, num=self.Nsteps+1, endpoint=True)
        sqrt_dt = math.sqrt(self.dt)

        # S array (Nb of time steps x Nb of simulations)
        self.S = np.empty(shape=(self.Nsteps + 1, self.Nsimul))
        self.S[0,:] = self.S0

        # The drift mu : r - q
        mu = self.r - self.q
        # Spot simulations from t to t + dt
        for j in range(self.Nsteps) :
            self.S[j+1,:] = self.S[j,:] * np.exp((mu - 0.5 * self.vol**2) * self.dt + self.vol * sqrt_dt * normal[j,:])

    def mc_price(self, flavor, K) :
        """
        The Monte-Carlo American Price.
        """
        if self.set_data == False :
            print("Please set option data first : \"set_option_data\" method.")
            return None

        if not hasattr(self, 'S') :
            print("Please run spot simulations first : \"set_path\" method")
            return None

        # Option payoff at maturity
        phi = 1 if flavor.upper() == "CALL" else -1
        exercise_prices = np.maximum(phi * (self.S - K), 0)
        df = math.exp(-self.r * self.dt)

        # Backward loop
        prices = np.empty(shape=(self.Nsteps + 1, self.Nsimul))
        prices[-1,:] = exercise_prices[-1,:]
        for j in range(self.Nsteps - 1, 0, -1):
            # Polynomial Regression (19 degree - 20 coefficients)
            regression = np.polyfit(self.S[j, :], df * prices[j + 1,:], 19)
            # Continuation Value
            continuation_prices = np.polyval(regression, self.S[j, :])
            # Possibility of early exercise
            prices[j,:] = np.where(exercise_prices[j,:] >= continuation_prices, exercise_prices[j,:], df * prices[j + 1,:])

        prices[0,:] = df * prices[1,:]
        return max(np.mean(prices[0,:]), phi * (self.S0 - K))

## Application

Let us apply and compare the three pricing methods :


In [4]:
# Data
N = 10000.
S0 = 120.
T = 0.5
vol = 0.35
r = 0.03
q = 0.01

# Models Initialization
binomial = binomial_model()
binomial.set_option_data(S0, T, vol, r, q)
baw = baw_model()
baw.set_option_data(S0, T, vol, r, q)
mc = ls_mc()
mc.set_option_data(S0, T, vol, r, q)
mc.set_path()

data ={"Contract Size" : N, "Spot Price" : S0, "Maturity" : T, "ATM Implied Volatility" : vol, "ZC Rate" : r, "Dividend Yield" : q}
print("Input Data : ", data)

print("\n*************************************************")
# Scenario 1 : Strike = 90% * Spot
K = S0 * 0.9
print("Scenario 1  : Strike = 90% * Spot")
print("-------------------------------------------------")
## Put Options
print("The Binomial Put Price : ", round(N * binomial.binomial_price("Put", K), 2))
print("The BAW Put Price : ", round(N * baw.baw_price("Put", K), 2))
print("The LS Monte-Carlo Put Price : ", round(N * mc.mc_price("Put", K), 2))
print("-------------------------------------------------")
## Call Options
print("The Binomial Call Price : ", round(N * binomial.binomial_price("Call", K), 2))
print("The BAW Call Price : ", round(N * baw.baw_price("Call", K), 2))
print("The LS Monte-Carlo Call Price : ", round(N * mc.mc_price("Call", K), 2))
print("*************************************************")
# Scenario 2 : Strike = 110% * Spot
K = S0 * 1.1
print("Scenario 2  : Strike = 110% * Spot")
print("-------------------------------------------------")
## Put Options
print("The Binomial Put Price : ", round(N * binomial.binomial_price("Put", K), 2))
print("The BAW Put Price : ", round(N * baw.baw_price("Put", K), 2))
print("The LS Monte-Carlo Put Price : ", round(N * mc.mc_price("Put", K), 2))
print("-------------------------------------------------")
## Call Options
print("The Binomial Call Price : ", round(N * binomial.binomial_price("Call", K), 2))
print("The BAW Call Price : ", round(N * baw.baw_price("Call", K), 2))
print("The LS Monte-Carlo Call Price : ", round(N * mc.mc_price("Call", K), 2))
print("*************************************************")

Input Data :  {'Contract Size': 10000.0, 'Spot Price': 120.0, 'Maturity': 0.5, 'ATM Implied Volatility': 0.35, 'ZC Rate': 0.03, 'Dividend Yield': 0.01}

*************************************************
Scenario 1  : Strike = 90% * Spot
-------------------------------------------------
The Binomial Put Price :  58361.9
The BAW Put Price :  58402.83
The LS Monte-Carlo Put Price :  57739.43
-------------------------------------------------
The Binomial Call Price :  188019.04
The BAW Call Price :  188020.21
The LS Monte-Carlo Call Price :  187315.84
*************************************************
Scenario 2  : Strike = 110% * Spot
-------------------------------------------------
The Binomial Put Price :  185263.68
The BAW Put Price :  184908.87
The LS Monte-Carlo Put Price :  184704.95
-------------------------------------------------
The Binomial Call Price :  76843.02
The BAW Call Price :  76842.65
The LS Monte-Carlo Call Price :  76219.1
********************************************